## Data label poisoning

In [29]:
import torch
from PIL import Image
import torchvision
import torchvision.transforms as transforms
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Using {device} for inference')

Using cuda for inference


In [2]:
!pip install torchviz
!pip install torchsummary
!pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4131 sha256=6530fb0704c495771cb20d633fee3f8440717d90a2a80536229c48b6bd096329
  Stored in directory: /root/.cache/pip/wheels/4c/97/88/a02973217949e0db0c9f4346d154085f4725f99c4f15a87094
Successfully built torchviz
  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=b29d4d734d7bc143ffc344e5b2916eab40e1449022772eb4a1059f490127ddc8
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch


In [43]:
model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_widese_b4', pretrained=False)
model.eval().to(device)
import torch.optim as optim
import torch.nn as nn
num_epochs = 100
learning_rate = 0.0005
weight_decay = 0.0001
momentum = 0.01
optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay, momentum=momentum)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[20, 40])
criterion = nn.CrossEntropyLoss()
checkpoint = torch.load("/kaggle/input/ef-net/ef_net_78.pth")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

epoch = checkpoint['epoch']
loss = checkpoint['loss']
accuracy = checkpoint['accuracy']
average_loss_test = checkpoint['test_accuracy']
average_loss = checkpoint['test_loss']
learning_rate = checkpoint['learning_rate']

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

import torchvision.datasets as datasets
normalize= transforms.Normalize(mean=[0.49186882, 0.48265398, 0.44717732], std=[0.24697122, 0.24338895, 0.2615926 ])
transform = transforms.Compose([
        transforms.Resize(64),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.05, contrast=0.05, saturation=0.05, hue=0.05),
        # transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.49186882, 0.48265398, 0.44717732],std=[0.24697122, 0.24338895, 0.2615926 ])
    ])

batch_size = 64
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(
        datasets.CIFAR10(root='./data', train=True, transform=transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomCrop(32, 4),
            transforms.ToTensor(),
            normalize,
        ]), download=True),
        batch_size=batch_size, shuffle=True, pin_memory=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(
        datasets.CIFAR10(root='./data', train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            normalize,
        ]), download=True),
        batch_size=batch_size, shuffle=True, pin_memory=True)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [87]:
batch_size = 64
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(
        datasets.CIFAR10(root='./data', train=True, transform=transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomCrop(32, 4),
            transforms.ToTensor(),
            normalize,
        ]), download=True),
        batch_size=batch_size, shuffle=True, pin_memory=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(
        datasets.CIFAR10(root='./data', train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            normalize,
        ]), download=True),
        batch_size=batch_size, shuffle=True, pin_memory=True)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


Accuracy before label poisoning

In [4]:
import numpy as np

model.eval().to(device)
correct_predictions = np.zeros(len(classes))
total_samples = np.zeros(len(classes))

with torch.no_grad():
    for (inputs, targets) in testloader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        _, predicted = outputs.max(1)

        for label, prediction in zip(targets, predicted):
            if label == prediction:
                correct_predictions[label] += 1
            total_samples[label] += 1

for i, classname in enumerate(classes):
    accuracy = 100 * correct_predictions[i] / total_samples[i]
    print(f'Accuracy on test data {classname:5s} : {accuracy:.2f}%')

Accuracy on test data plane : 84.50%
Accuracy on test data car   : 90.90%
Accuracy on test data bird  : 74.20%
Accuracy on test data cat   : 68.50%
Accuracy on test data deer  : 81.10%
Accuracy on test data dog   : 70.90%
Accuracy on test data frog  : 87.40%
Accuracy on test data horse : 85.90%
Accuracy on test data ship  : 89.80%
Accuracy on test data truck : 89.40%


## Labet poisoning

In [5]:
import torchvision.datasets as datasets

train_labels = np.array(trainset.targets)

target_class = 3
percentage_to_change = 50
#new_label = 5

all_labels = set(range(10))
exclude_label = {target_class}
possible_new_labels = list(all_labels - exclude_label)

num_to_change = int((percentage_to_change / 100) * np.sum(train_labels == target_class))
indices_to_change = np.random.choice(np.where(train_labels == target_class)[0], num_to_change, replace=False)


print("Index\tOriginal Label\tNew Label")
for index in indices_to_change:
    new_label = np.random.choice(possible_new_labels)
    print(f"{index}\t{train_labels[index]}\t{new_label}")
    train_labels[index] = new_label

train_labels[indices_to_change] = new_label
trainset.targets = list(train_labels)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=False)

Index	Original Label	New Label
9135	3	5
28050	3	5
46081	3	8
44677	3	5
30270	3	7
28631	3	5
24842	3	0
23286	3	6
78	3	2
13349	3	1
4308	3	6
74	3	8
47475	3	1
4760	3	9
20882	3	6
9398	3	2
46627	3	0
43554	3	9
32392	3	1
23761	3	1
207	3	8
21	3	1
32046	3	8
21415	3	4
11976	3	8
49131	3	0
39461	3	6
39021	3	9
12370	3	5
17660	3	2
19140	3	2
23942	3	4
27722	3	8
2197	3	7
43336	3	0
46204	3	2
22171	3	8
48638	3	7
27250	3	8
46697	3	7
12776	3	6
36145	3	0
10958	3	0
49825	3	2
6343	3	9
12428	3	9
28121	3	2
4835	3	4
21731	3	6
21948	3	2
9842	3	0
49222	3	0
15799	3	6
23353	3	8
46503	3	7
20215	3	2
46812	3	5
43018	3	2
16228	3	5
1700	3	2
6607	3	7
29213	3	0
33642	3	4
9479	3	6
23594	3	1
3248	3	9
40232	3	8
37194	3	0
21744	3	4
28523	3	9
6200	3	5
49280	3	1
40160	3	2
22560	3	4
39616	3	2
13013	3	1
30777	3	1
22549	3	4
6208	3	8
38066	3	4
44635	3	7
8908	3	5
691	3	8
16405	3	6
6557	3	0
29848	3	4
29096	3	9
25545	3	5
24749	3	6
45091	3	9
21106	3	8
43701	3	4
7006	3	7
6637	3	0
18482	3	9
26912	3	0
2876	3	5
9405	3	2
22294	3	1
38984	3	1
40

## TEST

In [6]:
####TEST ONLY
test_labels = np.array(testset.targets)

num_to_change_test = int((percentage_to_change / 100) * np.sum(test_labels == target_class))
indices_to_change_test = np.random.choice(np.where(test_labels == target_class)[0], num_to_change_test, replace=False)

print("Index\tOriginal Label\tNew Label")
for index in indices_to_change_test:
    new_label = np.random.choice(possible_new_labels)
    print(f"{index}\t{test_labels[index]}\t{new_label}")
    test_labels[index] = new_label

testset.targets = list(test_labels)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)


Index	Original Label	New Label
7016	3	0
5336	3	9
4785	3	4
8433	3	4
5490	3	7
2923	3	5
6283	3	7
8251	3	8
2606	3	2
2937	3	7
4093	3	4
9576	3	2
3700	3	8
882	3	2
9992	3	7
5313	3	9
7717	3	8
77	3	5
558	3	1
8337	3	0
5655	3	2
4495	3	4
5779	3	2
8141	3	6
7291	3	8
4407	3	2
2422	3	5
3013	3	0
1346	3	5
7349	3	1
6009	3	0
2646	3	6
2878	3	8
7489	3	7
9562	3	6
998	3	6
6021	3	7
3183	3	2
2855	3	6
4708	3	9
3867	3	1
5290	3	1
9093	3	6
8295	3	1
5632	3	6
3125	3	8
8231	3	7
7415	3	0
9460	3	0
251	3	9
1275	3	2
7004	3	8
6680	3	4
5424	3	8
9647	3	5
786	3	1
4864	3	9
8739	3	8
5991	3	8
4467	3	8
6912	3	0
7961	3	4
1660	3	8
608	3	1
9043	3	6
2049	3	0
9089	3	9
7569	3	2
3280	3	4
453	3	8
6710	3	8
3634	3	7
3654	3	9
3628	3	5
2840	3	4
586	3	5
1377	3	2
1088	3	4
5671	3	8
6026	3	2
8196	3	0
106	3	2
9308	3	0
2832	3	6
5752	3	9
6311	3	9
945	3	6
1373	3	0
3594	3	2
2173	3	9
3249	3	2
4637	3	6
8996	3	6
8548	3	5
8077	3	9
6990	3	0
7212	3	9
8665	3	7
599	3	5
7970	3	7
1334	3	0
8435	3	0
5700	3	6
5213	3	4
2759	3	8
3067	3	5
5176	3	0
5191	3	2
4306	3	9
6

Accuracy after data poisoning

In [7]:
import numpy as np

model.eval().to(device)
correct_predictions = np.zeros(len(classes))
total_samples = np.zeros(len(classes))

with torch.no_grad():
    for (inputs, targets) in testloader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        _, predicted = outputs.max(1)

        for label, prediction in zip(targets, predicted):
            if label == prediction:
                correct_predictions[label] += 1
            total_samples[label] += 1

for i, classname in enumerate(classes):
    accuracy = 100 * correct_predictions[i] / total_samples[i]
    print(f'Accuracy on test data {classname:5s} : {accuracy:.2f}%')

Accuracy on test data plane : 0.19%
Accuracy on test data car   : 0.00%
Accuracy on test data bird  : 15.70%
Accuracy on test data cat   : 75.80%
Accuracy on test data deer  : 1.88%
Accuracy on test data dog   : 2.17%
Accuracy on test data frog  : 0.00%
Accuracy on test data horse : 0.00%
Accuracy on test data ship  : 0.00%
Accuracy on test data truck : 1.61%


## Implementing SimCLR

Loss function

In [8]:
import torch.nn as nn

def nt_xent_loss(z1, z2, temperature=0.5):
    z1_norm = F.normalize(z1, dim=1)
    z2_norm = F.normalize(z2, dim=1)

    sim_matrix = torch.einsum('ik,jk->ij', z1_norm, z2_norm)
    neg_sim = -sim_matrix
    log_prob = F.log_softmax(neg_sim / temperature, dim=1)
    pos_label = torch.arange(z1.shape[0], device=z1.device).long().unsqueeze(1)
    log_prob_pos = torch.gather(log_prob, 1, pos_label)

    loss = -torch.mean(log_prob_pos)
    return loss



Data Augmentations

In [31]:

augment_1 = transforms.Compose([
    transforms.RandomResizedCrop(size=224),
    transforms.RandomHorizontalFlip(p=0.5),
])

augment_2 = transforms.Compose([
    transforms.RandomResizedCrop(size=224),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomGrayscale(p=0.2),
])

 Model and Projection Head:

In [10]:
import torch.nn as nn

class ProjectionHead(nn.Module):
    def __init__(self, in_features, hidden_dim, out_features):
        super(ProjectionHead, self).__init__()
        self.fc1 = nn.Linear(in_features, hidden_dim)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(hidden_dim, out_features)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [37]:
model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_widese_b4', pretrained=False)
model.eval().to(device)
model = torch.load("/kaggle/input/ef-net/ef_net_78.pth")

epoch = checkpoint['epoch']
loss = checkpoint['loss']
accuracy = checkpoint['accuracy']
average_loss_test = checkpoint['test_accuracy']
average_loss = checkpoint['test_loss']
learning_rate = checkpoint['learning_rate']
# model.classifier = nn.Identity()

Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


## SimCLR training

TypeError: Tensor is not a torch image.

In [77]:
import torch
from torchvision import transforms
from torchvision.transforms import ToPILImage, ToTensor
from torchvision.transforms.functional import to_tensor, to_pil_image
import torch.nn.functional as F
model.to(device)

def nt_xent_loss(z1, z2, temperature=0.3):
    z1_norm = F.normalize(z1, dim=1)
    z2_norm = F.normalize(z2, dim=1)

    sim_matrix = torch.einsum('ik,jk->ij', z1_norm, z2_norm)
    neg_sim = -sim_matrix
    log_prob = F.log_softmax(neg_sim / temperature, dim=1)
    pos_label = torch.arange(z1.shape[0], device=z1.device).long().unsqueeze(1)
    log_prob_pos = torch.gather(log_prob, 1, pos_label)

    loss = -torch.mean(log_prob_pos)
    return loss
    

class ProjectionHead(nn.Module):
    def __init__(self, in_features, hidden_dim, out_features):
        super(ProjectionHead, self).__init__()
        self.fc1 = nn.Linear(in_features, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.relu = nn.ReLU(inplace=True)
        # Example: Adding an additional layer
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, out_features)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

    

to_pil_image = transforms.ToPILImage()
    
augment_1 = transforms.Compose([
    transforms.RandomResizedCrop(size=128),
    transforms.RandomHorizontalFlip(p=0.5),
])

augment_2 = transforms.Compose([
    transforms.RandomResizedCrop(size=128),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomGrayscale(p=0.2),
])
learning_rate=0.001
optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay, momentum=momentum)

projection_head = ProjectionHead(1000, 128, 128).to(device)
# train(epochs=100, trainloader=trainloader, optimizer=optimizer, scheduler=scheduler)


from tqdm import tqdm
epochs = 50
num_epochs = epochs

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    total_batches = 0
    correct_pairs = 0

    train_loader = tqdm(trainloader, total=len(trainloader), desc=f'Epoch [{epoch + 1}/{num_epochs}]')

    for _, (images, _) in enumerate(train_loader):
        images = images.to(device)
        augmented_images = []
        for image in images:
            image_pil = to_pil_image(image.cpu())
            img1 = augment_1(image_pil)
            img2 = augment_2(image_pil)
            img1 = to_tensor(img1).to(device)
            img2 = to_tensor(img2).to(device)
            augmented_images.extend([img1, img2])

        augmented_images = torch.stack(augmented_images, dim=0)

        h1, h2 = model(augmented_images[::2]), model(augmented_images[1::2])
        z1 = projection_head(h1)
        z2 = projection_head(h2)

        loss = nt_xent_loss(z1, z2, temperature=0.3)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Compute similarity for positive pairs and compare against negatives for a rough "accuracy"
        sim_pos = F.cosine_similarity(z1, z2)
        sim_neg_mean = (torch.sum(torch.exp(-loss)) - torch.sum(torch.exp(sim_pos))) / (len(sim_pos) * (len(sim_pos) - 1))
        correct_pairs += torch.sum(sim_pos > sim_neg_mean).item()

        total_loss += loss.item()
        total_batches += 1

        current_average_loss = total_loss / total_batches
        current_accuracy = correct_pairs / (total_batches * images.size(0))
        train_loader.set_postfix(loss=current_average_loss, accuracy=current_accuracy)

    average_loss = total_loss / total_batches

    scheduler.step()

Epoch [1/50]:   1%|▏         | 10/782 [00:07<09:55,  1.30it/s, accuracy=0.991, loss=4.26]


KeyboardInterrupt: 

In [88]:
import numpy as np

model.eval().to(device)
correct_predictions = np.zeros(len(classes))
total_samples = np.zeros(len(classes))

with torch.no_grad():
    for (inputs, targets) in testloader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        _, predicted = outputs.max(1)

        for label, prediction in zip(targets, predicted):
            if label == prediction:
                correct_predictions[label] += 1
            total_samples[label] += 1

for i, classname in enumerate(classes):
    accuracy = 100 * correct_predictions[i] / total_samples[i]
    print(f'Accuracy on test data {classname:5s} : {accuracy:.2f}%')

Accuracy on test data plane : 0.20%
Accuracy on test data car   : 99.40%
Accuracy on test data bird  : 0.10%
Accuracy on test data cat   : 0.90%
Accuracy on test data deer  : 1.70%
Accuracy on test data dog   : 16.00%
Accuracy on test data frog  : 0.00%
Accuracy on test data horse : 0.00%
Accuracy on test data ship  : 0.70%
Accuracy on test data truck : 0.00%
